In [16]:
#env: equake_env (Python 3.9)
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [17]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [18]:
#create and connect to a sqlite3 db
path = "../Database/db/earthquake_db"
conn = sqlite3.connect(path)

Adding 1 month of data

In [19]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-06-26T20:31:39.635Z,64.809400,-148.808800,8.70,0.80,ml,NaN,NaN,NaN,0.1900,...,2022-06-26T20:34:02.463Z,"27 km NNE of Four Mile Road, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
1,2022-06-26T20:20:53.425Z,38.138900,-118.004800,6.30,1.00,ml,11.0,142.81,0.036000,0.1658,...,2022-06-26T20:24:44.429Z,"29 km SSE of Mina, Nevada",earthquake,NaN,0.50,0.220000,7.0,automatic,nn,nn
2,2022-06-26T20:18:02.730Z,35.706833,-117.477500,11.25,1.04,ml,12.0,197.00,0.120700,0.1400,...,2022-06-26T20:21:41.957Z,"10km SW of Searles Valley, CA",earthquake,0.57,0.93,0.136000,9.0,automatic,ci,ci
3,2022-06-26T20:16:22.670Z,33.985500,-118.353167,13.02,1.38,ml,28.0,58.00,0.008693,0.2700,...,2022-06-26T20:27:28.800Z,"1km SSW of View Park-Windsor Hills, CA",earthquake,0.47,0.63,0.184000,29.0,automatic,ci,ci
4,2022-06-26T19:53:28.242Z,38.172900,-117.828000,4.30,0.90,ml,8.0,185.49,0.109000,0.1276,...,2022-06-26T19:57:32.002Z,"34 km SE of Mina, Nevada",earthquake,NaN,3.60,0.380000,5.0,automatic,nn,nn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8933,2022-05-27T21:14:04.000Z,33.973833,-116.790667,18.08,1.46,ml,43.0,33.00,0.062790,0.1300,...,2022-05-27T22:42:45.580Z,"6km N of Cabazon, CA",earthquake,0.16,0.39,0.119000,23.0,reviewed,ci,ci
8934,2022-05-27T21:09:32.280Z,19.382000,-155.493500,8.09,1.84,ml,36.0,57.00,NaN,0.1500,...,2022-05-27T21:15:04.140Z,"19 km N of Pāhala, Hawaii",earthquake,0.37,0.66,5.200000,3.0,automatic,hv,hv
8935,2022-05-27T20:59:14.910Z,41.504833,-112.170500,4.42,1.08,ml,15.0,64.00,0.104300,0.1000,...,2022-05-27T21:21:01.420Z,"7 km SW of Corinne, Utah",earthquake,0.30,1.83,0.059000,2.0,reviewed,uu,uu
8936,2022-05-27T20:48:47.495Z,63.601100,-149.452800,107.30,2.00,ml,NaN,NaN,NaN,0.4300,...,2022-05-27T21:00:15.300Z,"30 km WSW of Denali Park, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak


In [20]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

8938

In [ ]:
#creates/replaces database to add new data to json file
#=============
#PostgreSQL
df.to_sql('earthquake_update', engine, if_exists='replace', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_update', conn, if_exists='append', index=False)

Load from database to update with new rows and remove duplicate rows

In [21]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73140,2022-05-27T21:14:04.000Z,33.973833,-116.790667,18.08,1.46,ml,43.0,33.00,0.06279,0.1300,...,2022-05-27T22:42:45.580Z,"6km N of Cabazon, CA",earthquake,0.16,0.39,0.119000,23.0,reviewed,ci,ci
73141,2022-05-27T21:09:32.280Z,19.382000,-155.493500,8.09,1.84,ml,36.0,57.00,NaN,0.1500,...,2022-05-27T21:15:04.140Z,"19 km N of Pāhala, Hawaii",earthquake,0.37,0.66,5.200000,3.0,automatic,hv,hv
73142,2022-05-27T20:59:14.910Z,41.504833,-112.170500,4.42,1.08,ml,15.0,64.00,0.10430,0.1000,...,2022-05-27T21:21:01.420Z,"7 km SW of Corinne, Utah",earthquake,0.30,1.83,0.059000,2.0,reviewed,uu,uu
73143,2022-05-27T20:48:47.495Z,63.601100,-149.452800,107.30,2.00,ml,NaN,NaN,NaN,0.4300,...,2022-05-27T21:00:15.300Z,"30 km WSW of Denali Park, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak


In [22]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73140,2022-05-27T21:14:04.000Z,33.973833,-116.790667,18.08,1.46,ml,43.0,33.00,0.06279,0.1300,...,2022-05-27T22:42:45.580Z,"6km N of Cabazon, CA",earthquake,0.16,0.39,0.119000,23.0,reviewed,ci,ci
73141,2022-05-27T21:09:32.280Z,19.382000,-155.493500,8.09,1.84,ml,36.0,57.00,NaN,0.1500,...,2022-05-27T21:15:04.140Z,"19 km N of Pāhala, Hawaii",earthquake,0.37,0.66,5.200000,3.0,automatic,hv,hv
73142,2022-05-27T20:59:14.910Z,41.504833,-112.170500,4.42,1.08,ml,15.0,64.00,0.10430,0.1000,...,2022-05-27T21:21:01.420Z,"7 km SW of Corinne, Utah",earthquake,0.30,1.83,0.059000,2.0,reviewed,uu,uu
73143,2022-05-27T20:48:47.495Z,63.601100,-149.452800,107.30,2.00,ml,NaN,NaN,NaN,0.4300,...,2022-05-27T21:00:15.300Z,"30 km WSW of Denali Park, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak


In [23]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [24]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [25]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time  latitude  longitude   depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.10  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.30  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.00  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.30  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.90  1.5      ml   
...                         ...       ...        ...     ...  ...     ...   
64209  2022-06-21T13:47:59.490Z  -28.2245  -177.0944   41.17  4.1      mb   
64210  2022-06-21T13:37:09.523Z  -15.0919  -177.1751  363.17  4.2      mb   
64211  2022-06-21T13:27:10.754Z   27.8278   142.4459   25.50  4.2      mb   
64212  2022-06-20T15:59:38.509Z  -17.8954  -178.3020  599.16  4.2      mb   
64213  2022-06-20T15:07:33.761Z  -10.1190   161.4266   99.55  4.4      mb   

        nst     gap   dmin     rms  ...                   updated  \
0     

In [26]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time  latitude  longitude   depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.10  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.30  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.00  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.30  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.90  1.5      ml   
...                         ...       ...        ...     ...  ...     ...   
64202  2022-06-21T13:47:59.490Z  -28.2245  -177.0944   41.17  4.1      mb   
64203  2022-06-21T13:37:09.523Z  -15.0919  -177.1751  363.17  4.2      mb   
64204  2022-06-21T13:27:10.754Z   27.8278   142.4459   25.50  4.2      mb   
64205  2022-06-20T15:59:38.509Z  -17.8954  -178.3020  599.16  4.2      mb   
64206  2022-06-20T15:07:33.761Z  -10.1190   161.4266   99.55  4.4      mb   

        nst     gap   dmin     rms  ...                   updated  \
0     

In [27]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

207

In [28]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

64207

In [29]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [30]:
# Close connection to SQLite database
conn.close()
connection.close()